In [1]:
import polars as pl

In [2]:
df = pl.read_csv("D:\Books\instruments.csv")

In [3]:
df.filter(
    (pl.col("exchange") == "NSE") & (pl.col("instrument_type") == "EQ") & (pl.col("name").is_not_null()) & ~pl.col("tradingsymbol").str.starts_with("NIFTY")
    ).select(
    "tradingsymbol",
    "name",
).get_columns()[0]

tradingsymbol
str
"""HANGSENG BEES-NAV"""
"""INDIA VIX"""
"""GOLDSTAR-SM"""
"""AARTIIND"""
"""ABB"""
…
"""NBCC"""
"""LYPSAGEMS"""
"""ORBTEXP"""


In [4]:
df.head()

instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
i64,i64,str,str,i64,str,f64,f64,i64,str,str,str
261799430,1022654,"""EURINR24JULFUT""","""EURINR""",0,"""2024-07-29""",0.0,0.0025,1,"""FUT""","""BCD-FUT""","""BCD"""
262676230,1026079,"""EURINR24AUGFUT""","""EURINR""",0,"""2024-08-28""",0.0,0.0025,1,"""FUT""","""BCD-FUT""","""BCD"""
263556614,1029518,"""EURINR24SEPFUT""","""EURINR""",0,"""2024-09-26""",0.0,0.0025,1,"""FUT""","""BCD-FUT""","""BCD"""
264741126,1034145,"""EURINR24OCTFUT""","""EURINR""",0,"""2024-10-29""",0.0,0.0025,1,"""FUT""","""BCD-FUT""","""BCD"""
265969926,1038945,"""EURINR24NOVFUT""","""EURINR""",0,"""2024-11-27""",0.0,0.0025,1,"""FUT""","""BCD-FUT""","""BCD"""


In [5]:
import polars as pl

In [6]:
c1 = pl.read_csv("listing_dates.csv")
c2 = pl.read_csv("listing_dates_correct.csv")
c = c1.vstack(c2)

In [7]:
c.shape

(1896, 2)

In [8]:
data = df.lazy().filter(
    (pl.col("exchange") == "NSE") &
    (pl.col("instrument_type") == "EQ") & 
    (pl.col("name").is_not_null())
).select(
    "tradingsymbol",
    "name",
    "instrument_type",
    "segment",
    "exchange"
).collect()

In [9]:
data.shape

(2603, 5)

In [10]:
res = data.join(c, how="left", left_on="tradingsymbol", right_on="symbol")

In [11]:
null_res = res.select("tradingsymbol").get_columns()[0].to_list()

In [18]:
c.filter(~pl.col("symbol").is_in(null_res)).write_excel("left_listing_dates.xlsx")

In [13]:
res.select(
    "name",
    "tradingsymbol",
    "exchange",
    "instrument_type",
    "date_of_listing"
).filter(
    pl.col("date_of_listing").is_not_null()
)

name,tradingsymbol,exchange,instrument_type,date_of_listing
str,str,str,str,str
"""AARTI INDUSTRIES""","""AARTIIND""","""NSE""","""EQ""","""08-Feb-1995"""
"""ABB INDIA""","""ABB""","""NSE""","""EQ""","""08-Feb-1995"""
"""STYRENIX PERFORMANCE""","""STYRENIX""","""NSE""","""EQ""","""29-Mar-1995"""
"""ACC""","""ACC""","""NSE""","""EQ""","""20-Nov-1996"""
"""ADANI ENTERPRISES""","""ADANIENT""","""NSE""","""EQ""","""04-Jun-1997"""
…,…,…,…,…
"""NBCC (INDIA)""","""NBCC""","""NSE""","""EQ""","""12-Apr-2012"""
"""LYPSA GEMS & JEWEL""","""LYPSAGEMS""","""NSE""","""EQ""","""05-Nov-2013"""
"""ORBIT EXPORTS""","""ORBTEXP""","""NSE""","""EQ""","""05-Nov-2013"""


In [17]:
res.with_columns(
    pl.col("date_of_listing").str.strptime(pl.Datetime, format = "%d-%b-%Y").alias("listing_date")
).select(
    pl.exclude("date_of_listing")
).write_excel("nse_listing_date.xlsx")

In [16]:
%pip install xlsxwriter


  Obtaining dependency information for xlsxwriter from https://files.pythonhosted.org/packages/a7/ea/53d1fe468e63e092cf16e2c18d16f50c29851242f9dd12d6a66e0d7f0d02/XlsxWriter-3.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/159.9 kB ? eta -:--:--
   ----------------------- ---------------- 92.2/159.9 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 159.9/159.9 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
